In [32]:
import snowflake.connector
import requests
import json
import os

def get_snowflake_token() -> str:
    url = os.getenv("SNOWFLAKE_TOKEN_GENERATOR_URL") + "/token"
    data = {
        "okta_username": os.getenv("OKTA_USERNAME"),
        "okta_password": os.getenv("OKTA_PASSWORD"),
        "snowflake_role": os.getenv("SNOWFLAKE_ROLE"),
    }
    response = requests.post(url=url, data=json.dumps(data))
    if response.ok:
        return response.json()["token"]
    raise ValueError("Error getting the token. Check your credentials.")

# print(os.environ['SNOWFLAKE_USER'], os.environ['SNOWFLAKE_PASSWORD'])
con = snowflake.connector.connect(
    user= f"{os.getenv('OKTA_USERNAME')}@earnest.com",
    # password=os.environ['SNOWFLAKE_PASSWORD'],
    account=os.environ['SNOWFLAKE_ACCOUNT'],
    role=os.environ['SNOWFLAKE_ROLE'],
    warehouse=os.environ['SNOWFLAKE_WAREHOUSE'],
    authenticator="externalbrowser",
    # token=get_snowflake_token(),
)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://meetearnest.okta.com/app/snowflake/exkny4h8a73d8LUPZ0x7/sso/saml?SAMLRequest=lZLRbtowFIZfJfKuEzuBEWYBFSuqmoq1FAKtuHOTA1gkdubjNNCnnwll6i5aaXeR%2Ff3OZ%2F9ncHUoC%2B8VDEqthiQMGPFAZTqXajsky%2FTG7xMPrVC5KLSCITkCkqvRAEVZVHxc252aw%2B8a0HruIIW83RiS2iiuBUrkSpSA3GZ8Mf415VHAeGW01ZkuyIfI1wmBCMY6w0skR%2Bn0dtZWnNKmaYKmE2izpRFjjLIf1FEn5NuFP7g7fcKHlHVPvCMcPnt3%2BynV%2BQm%2B0no5Q8hv03Tmzx4WKfHGF9VrrbAuwSzAvMoMlvPpWQCdwXI1jlnE4qBGHwRaPwxQ6WZTiD1kuqxq644N3BfdQE4LvZXu5slkSKq9zLN4%2FtabPDV3z9uX8Pax3EF1bBZ3b%2Bn0%2BR4m%2BePaJMkTrI7TUGfEW12qjU7VJog1JOpUqHVLLOr6IfOjXspC%2Fj3mLAo6vXBNvIkrVCph2%2BTFugSwIIxye4HeW9EKiqqif90pHPbq2N31RdzJ%2B9PlbM0OMUXU9NQxOY8NbyXM6L8fY0A%2Fxt9H8N61kkxmupDZ0bvRphT289LCIGxXZO5vWpRDKWQxznMDiK68otDNtQFh3aRbUwOho%2FNf%2F5310R8%3D&Relay

In [33]:
cur = con.cursor().execute('select current_timestamp()')
cur.fetchone()

(datetime.datetime(2024, 10, 25, 18, 57, 43, 949000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),)

In [54]:
from datetime import datetime, date
datetime.today().date()

datetime.date(2024, 10, 25)

In [55]:
date(2024, 10, 5)

datetime.date(2024, 10, 5)

In [49]:
from enum import Enum
from datetime import datetime, timedelta
from typing import List

class PricingServiceLogTable(Enum):
    staging = 'RAW_TEST.STAGING__PRICING_SERVICE_V2__PUBLIC.VIEW_STAGING__PRICING_SERVICE_LOG'
    prod = 'RAW.PRICING_SERVICE_V2__PUBLIC.VIEW_PRICING_SERVICE_LOG'

def create_seed_id_query_from_pricing_service_log_table(
        seed_ids: List[str],
        table_name: PricingServiceLogTable = None,
        date: datetime.date = None
) -> str:
    quoted_seed_ids = map(lambda seed_id: f"'{seed_id}'", seed_ids)
    seed_ids_joined = ",".join(quoted_seed_ids)
    table_name = table_name if table_name else PricingServiceLogTable.staging
    date = date if date else (datetime.today().date() - timedelta(days=-7))
    return f"""
    with parsed_json as (select ID,
                                try_parse_json(OUTPUT_OBJ) as OUTPUT_OBJ,
                                try_parse_json(INPUT_OBJ)  as INPUT_OBJ,
                                CREATION_TS
                         from {table_name.value}
                         where product = 'slr-alloy' and CREATION_TS > '{date.isoformat()}')
    select
        id,
        VALUE:attributeValue::string as seed_id,
        OUTPUT_OBJ,
        INPUT_OBJ,
        CREATION_TS
    from parsed_json, lateral flatten(input => INPUT_OBJ:Earnest_Meta)
    where
        VALUE:attributeName = 'seedId' and VALUE:attributeValue::string in (
            {seed_ids_joined}
        )
    order by seed_id, CREATION_TS desc;
    """

print(create_seed_id_query_from_pricing_service_log_table(['14487949185843640404051566873587478166']))


    with parsed_json as (select ID,
                                try_parse_json(OUTPUT_OBJ) as OUTPUT_OBJ,
                                try_parse_json(INPUT_OBJ)  as INPUT_OBJ,
                                CREATION_TS
                         from RAW_TEST.STAGING__PRICING_SERVICE_V2__PUBLIC.VIEW_STAGING__PRICING_SERVICE_LOG
                         where product = 'slr-alloy' and CREATION_TS > '2024-11-01')
    select
        id,
        VALUE:attributeValue::string as seed_id,
        OUTPUT_OBJ,
        INPUT_OBJ,
        CREATION_TS
    from parsed_json, lateral flatten(input => INPUT_OBJ:Earnest_Meta)
    where
        VALUE:attributeName = 'seedId' and VALUE:attributeValue::string in (
            '14487949185843640404051566873587478166'
        )
    order by seed_id, CREATION_TS desc;
    


In [ ]:
query = create_seed_id_query_from_pricing_service_log_table(['14487949185843640404051566873587478166'], date=date(2024, 10, 1))
cur = con.cursor().execute(query)
cur.fetchall()

In [37]:
import pandas as pd
import polars as pl

query = """with parsed_json as (select ID,
                            try_parse_json(OUTPUT_OBJ) as OUTPUT_OBJ,
                            try_parse_json(INPUT_OBJ)  as INPUT_OBJ,
                            CREATION_TS
                     from RAW_TEST.STAGING__PRICING_SERVICE_V2__PUBLIC.VIEW_STAGING__PRICING_SERVICE_LOG
                     where product = 'slr-alloy' and CREATION_TS > '2024-10-01')
select
    id,
    VALUE:attributeValue::string as seed_id,
    OUTPUT_OBJ,
    INPUT_OBJ,
    CREATION_TS
from parsed_json, lateral flatten(input => INPUT_OBJ:Earnest_Meta)
where
    VALUE:attributeName = 'seedId' and VALUE:attributeValue::string in (
    '14487949185843640404051566873587478166'
    )
order by CREATION_TS desc
limit 10;"""

df = pd.read_sql(query, con)
df

/var/folders/m6/nxnqn8zx5xz8m277gc4b_plh0000gp/T/ipykernel_65437/2416432841.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con)


,ID,SEED_ID,OUTPUT_OBJ,INPUT_OBJ,CREATION_TS
0,576790,14487949185843640404051566873587478166,"{\n ""asset_lite_score_output"": {\n ""model_...","{\n ""Earnest_Data_Supplied"": [\n {\n ...",2024-10-23 17:08:23.885351
1,576789,14487949185843640404051566873587478166,None,"{\n ""Earnest_Data_Supplied"": [\n {\n ...",2024-10-23 17:08:16.637265
2,576787,14487949185843640404051566873587478166,"{\n ""asset_lite_score_output"": {\n ""model_...","{\n ""Earnest_Data_Supplied"": [\n {\n ...",2024-10-23 17:06:27.212295


In [44]:
df['has_output'] = ~df.OUTPUT_OBJ.isna()
df

,ID,SEED_ID,OUTPUT_OBJ,INPUT_OBJ,CREATION_TS,missing_output,has_output
0,576790,14487949185843640404051566873587478166,"{\n ""asset_lite_score_output"": {\n ""model_...","{\n ""Earnest_Data_Supplied"": [\n {\n ...",2024-10-23 17:08:23.885351,True,True
1,576789,14487949185843640404051566873587478166,None,"{\n ""Earnest_Data_Supplied"": [\n {\n ...",2024-10-23 17:08:16.637265,False,False
2,576787,14487949185843640404051566873587478166,"{\n ""asset_lite_score_output"": {\n ""model_...","{\n ""Earnest_Data_Supplied"": [\n {\n ...",2024-10-23 17:06:27.212295,True,True


In [51]:
df.dtypes

ID                         int64
SEED_ID                   object
OUTPUT_OBJ                object
INPUT_OBJ                 object
CREATION_TS       datetime64[ns]
missing_output              bool
has_output                  bool
dtype: object

In [58]:
import json
def parse_json(x):
    return json.loads(x) if x else x
    
df.OUTPUT_OBJ = df.OUTPUT_OBJ.apply(parse_json)

In [62]:
def has_prices(x):
    if x is None:
        return False
    return 'price_output' in x and (x['price_output'] is not None or x['price_output'] != [])

df.OUTPUT_OBJ.apply(has_prices)

0     True
1    False
2     True
Name: OUTPUT_OBJ, dtype: bool

In [65]:
df['parsed_input'] = df.INPUT_OBJ.apply(parse_json)

In [66]:

class ExperianField(str, Enum):
    rental_expense = "Rental Expenses (Earnest)"
    sole_mortgage_expense = "Sole Mortgage Expense (Earnest)"
    joint_mortgage_expense = "Joint Mortgage Expense (Earnest)"
    education_loan_balance = "Education Loan Balance"
    fico_score = "FICO Score"
    credit_card_balance = "Credit Card Balance"
    student_loan_expense = "Student Loan Expense (Earnest)"
    non_real_estate_expense = "Non-Real Estate/Education Expense (Earnest)"


class MetaField(str, Enum):
    seed_id = "seedId"
    applicant_role = "applicantRole"
    living_situation = "livingSituation"
    state = "state"
    loan_amount_approved_cents = "loan_amount_approved_cents"
    application_id = "applicationId"
    estimate_id = "estimateId"
    user_id = "userId"
    claimed_housing_payment = "claimedHousingPayment"
    expenses_claimed_housing_payment = "expenses_claimedHousingPayment"
    verified_housing_payment = "expenses_verifiedHousingPayment"
    verified_student_loan_balance = "expenses_verifiedStudentLoanBalance"
    rate_map_version = "quickScoreData_rateMapVersion"
    rate_map_tag = "quickScoreData_rateMapTag"
    rate_adjustment_data_name = "quickScoreData_rateAdjustmentData_name"
    rate_adjustment_data_amount = "quickScoreData_rateAdjustmentData_amount"
    claimed_loan_amount = "loanData_claimedLoanAmount"
    soft_approved_amount = "loanData_softApprovedAmount"
    primary_education_loan_balance = "primary_education_loan_balance"
    primary_score_curve = "loanData_scoreOutput"
    primary_price_curve = "loanData_priceOutput"
    primary_score_output = "primary_score_output"
    primary_price_output = "primary_price_output"


class EarnestOutputFields(str, Enum):
    verified_bonus = "verified_bonus_model"
    verified_checking = "verified_checking_model"
    verified_child_support_or_alimony = "verified_child_support_or_alimony_model"
    verified_disability = "verified_disability_model"
    verified_brokerage = "verified_brokerage_model"
    verified_housing_payment = "verified_housing_payment_model"
    verified_k1 = "verified_k1_model"
    verified_salary = "verified_salary_model"
    verified_rental = "verified_rental_model"
    verified_retirement = "verified_retirement_model"
    verified_social_security_or_pension = "verified_social_security_or_pension_model"
    verified_student_loan_balance = "verified_student_loan_balance_model"
    verified_investments = "verified_investments_model"
    verified_primary_student_loan_balance = "verified_primary_student_loan_balance"
    plaid_assets = "plaid_assets"


In [73]:
fields = {
    MetaField.state.value,
    MetaField.user_id.value,
    EarnestOutputFields.verified_bonus.value,
    ExperianField.fico_score.value
}

def is_not_null(val):
    return val is not None and val != 'null' and val != ''

def extract_fields(input_obj):
    result = {}
    for arr in ["Earnest_Meta", "Earnest_Output_Attributes", "Experian"]:
        for attribute in input_obj[arr]:
            if attribute['attributeName'] in fields and is_not_null(attribute['attributeValue']):
                result[attribute['attributeName']] = attribute['attributeValue']
    return result
    
df.parsed_input.apply(extract_fields)[0]

{'userId': '111729702966231', 'state': 'CA', 'FICO Score': '791'}